# Environment Set Up

In [13]:
import numpy as np
import random

In [14]:
maze = np.zeros((4,4)) # represents a gridworld in which each state, apart from the terminal state, has a value of 0. 
terminal_state = [(0,0), (3,3)]

## Concept Check

What is value? The sum of all future rewards that can be recieved from a given state s. These values are "long lasting. Tells us the long term effects of being in a certain state



## Iterative Policy Evaluation for Gridworld 

What is iterative policy evaluation? Finding the true value function for each state with full knowledge of the environment

In [15]:
# calculate the value of each state given the policy
def policy_eval(maze_values): 
  
    # WHY CAN'T I DO IT. 
                            
    #action = random.randint(0, (1/policy) - 1)
            # 0 = up, 1 = down, 2 = right, 3 = left. maybe put this in a dictionary or something
    # make a for loop for the states:
    
    new_values = maze_values.copy()
    
    for row in range(len(maze_values[0])): 
        for col in range(len(maze_values[1])):
            reward = -1 # -1 for up, down, left, right
            successor_val = 0
            initial_state_val = maze_values[row][col]
            
            if ((row, col) in terminal_state): 
                new_values[row][col] = 0
            else:
                # check left
                if (col - 1 < 0): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row][col - 1])
                # check right
                if (col + 1 >= len(maze_values[1])): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row][col + 1])
                # check up
                if (row - 1 < 0): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row - 1][col])
                # check down
                if (row + 1 >= len(maze_values[0])): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row+1][col])
                
                new_values[row][col] = 0.25 * successor_val
                
            
        
    return np.array(new_values) 
                
            
        
    

                

In [16]:
maze_values = np.zeros((4,4))

print(np.array(maze_values))

current_values = maze_values

for i in range(1000): 
    
    current_values = policy_eval(current_values)
    
    
print(current_values)



[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


# MC Prediction

How is MC Prediction different than Iterative Policy Evaluation? We do NOT have full knowledge of the environment. We find the true value function through episodes

What is First Visit Monte Carlo? Averaging returns for a state after the state's first occurence

In [218]:
def step(pos): 
    
    action = random.randint(0,3) # random policy
    successor_val = 0
    successor_state = pos
    reward = -1
    
    if action == 0: # check up
        if not (pos[0] - 1 < 0): 
            successor_state = (pos[0] - 1, pos[1])
    if action == 1: # check down
        if not (pos[0] + 1 > 3): 
            successor_state = (pos[0] + 1, pos[1])
    if action == 2: # check left
        if not (pos[1] - 1 < 0): 
            successor_state = (pos[0], pos[1] - 1)
    if action == 3: # check right
        if not (pos[1] + 1 > 3): 
            successor_state = (pos[0], pos[1] + 1)
    
    if successor_state in terminal_state: 
        reward = 0
        
    if pos in terminal_state:
        reward = 0
                               
    
    return pos, action, reward, successor_state


In [239]:
def first_visit_mc(num_iterations): 
    
    #maze_values = np.zeros((4,4))
    
    from collections import defaultdict
    
    state_rewards = defaultdict(int)
    returns = defaultdict(int)
    all_states = np.array([])
    
    for it in range(1, num_iterations + 1): 
        
        is_terminal = False
        
        initial_state = (random.randint(0,3), random.randint(0,3))
        states_visited = np.array([])
        while not is_terminal:
            initial_state, action, reward, successor_state = step(initial_state)
            states_visited = np.append(states_visited, [initial_state, action, reward, successor_state])
            initial_state = successor_state
            if initial_state in terminal_state: 
                states_visited = np.copy(states_visited.reshape(int(len(states_visited)/4), 4))
                all_states = states_visited
                is_terminal = True
        
        
        #print(all_states)
        for i in range(len(all_states)): 
        # need to get rid of all duplicates because that is throwing us off
        # step function needs to check if the pos is a terminal state so
            #print(all_states[i][0])
            if all_states[i][0] not in state_rewards.keys(): 
                state_rewards[all_states[i][0]] = np.sum(all_states[i:, 2])
       
        
   
    for state, reward in state_rewards.items(): 
            
        returns[state] = reward / num_iterations
        
        
   
    
    return returns, state_rewards
    
        
    
        
            
            
    # there is still something wrong with this        
            
            
            
    
    
    
    

In [244]:
returns = first_visit_mc(1000)
returns

(defaultdict(int,
             {(3, 2): 0.0,
              (2, 0): -0.016,
              (2, 1): -0.015,
              (3, 1): -0.014,
              (3, 0): -0.013,
              (1, 1): -0.003,
              (0, 1): -0.002,
              (0, 2): -0.014,
              (0, 3): -0.012,
              (1, 3): -0.011,
              (2, 3): 0.0,
              (0, 0): 0.0,
              (1, 0): -0.03,
              (1, 2): -0.011,
              (2, 2): -0.001,
              (3, 3): -0.01}),
 defaultdict(int,
             {(3, 2): 0,
              (2, 0): -16,
              (2, 1): -15,
              (3, 1): -14,
              (3, 0): -13,
              (1, 1): -3,
              (0, 1): -2,
              (0, 2): -14,
              (0, 3): -12,
              (1, 3): -11,
              (2, 3): 0,
              (0, 0): 0,
              (1, 0): -30,
              (1, 2): -11,
              (2, 2): -1,
              (3, 3): -10}))

In [203]:
# construct value grid

maze_values = np.zeros((4,4))

for i in range(maze_values.shape[0]):
    for j in range(maze_values.shape[1]):
        maze_values[i][j] = returns[(i,j)]
print(maze_values)
    


[[ -0.32  -8.22 -19.66 -41.38]
 [ -5.58 -12.86 -21.66 -27.14]
 [-13.8  -21.04 -19.02 -12.08]
 [-16.42 -23.32 -10.42  -0.98]]


In [189]:
# hold up, why aren't the terminal states 0. hhhh